# Tabular Playground Series - Feb 2022

## Importación de librerías a utilizar

In [1]:
import zipfile
import os

## Generación de DataFrame
### Primero descargo la información directo desde la API de Kaggle

In [2]:
conda install -c conda-forge kaggle

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
!kaggle competitions files -c tabular-playground-series-feb-2022

name                    size  creationDate         
---------------------  -----  -------------------  
train.csv                1GB  2022-01-31 13:54:40  
sample_submission.csv    3MB  2022-01-31 13:54:40  
test.csv               592MB  2022-01-31 13:54:40  


In [4]:
!kaggle competitions download -c tabular-playground-series-feb-2022


  0%|          | 0.00/267M [00:00<?, ?B/s]
  0%|          | 1.00M/267M [00:00<01:00, 4.59MB/s]
  1%|          | 2.00M/267M [00:00<00:49, 5.66MB/s]
  1%|1         | 3.00M/267M [00:00<00:56, 4.88MB/s]
  1%|1         | 4.00M/267M [00:00<00:50, 5.45MB/s]
  2%|1         | 5.00M/267M [00:01<00:54, 5.02MB/s]
  2%|2         | 6.00M/267M [00:01<00:51, 5.33MB/s]
  3%|2         | 7.00M/267M [00:01<00:59, 4.58MB/s]
  3%|2         | 8.00M/267M [00:01<01:07, 4.05MB/s]
  3%|3         | 9.00M/267M [00:02<01:02, 4.34MB/s]
  4%|3         | 10.0M/267M [00:02<00:54, 4.94MB/s]
  4%|4         | 11.0M/267M [00:02<00:51, 5.22MB/s]
  4%|4         | 12.0M/267M [00:02<00:48, 5.47MB/s]
  5%|4         | 13.0M/267M [00:02<00:47, 5.62MB/s]
  5%|5         | 14.0M/267M [00:02<00:48, 5.48MB/s]
  6%|5         | 15.0M/267M [00:03<00:50, 5.19MB/s]
  6%|5         | 16.0M/267M [00:03<00:54, 4.79MB/s]
  6%|6         | 17.0M/267M [00:03<00:56, 4.64MB/s]
  7%|6         | 18.0M/267M [00:03<00:51, 5.02MB/s]
  7%|7         | 19.

### Luego, la descomprimo, borro el zip y genero el DataFrame

In [5]:
with zipfile.ZipFile("tabular-playground-series-feb-2022.zip", "r") as zip_ref:
    zip_ref.extractall(r"C:\Users\alanp\Data Science\kaggle_tabular_playground\datasets")
os.remove("tabular-playground-series-feb-2022.zip")